# Spark 101

### 1. Create a spark data frame that contains your favorite programming languages.
 - The name of the column should be language

 - View the schema of the dataframe

 - Output the shape of the dataframe

 - Show the first 5 records in the dataframe


### 2. Load the mpg dataset as a spark dataframe.

 - Create 1 column of output that contains a message like the one below:
     
     - The 1999 audi a4 has a 4 cylinder engine
 
 
 - Transform the trans column so that it only contains either manual or auto
 

### 3. Load the tips dataset as a spark dataframe

   - What percentage of observations are smokers?
    
   - Create a column that contains the tip percentage
    
   - Calculate the average tip percentage for each combination of sex and smoker.
    

### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

   - Convert the temperatures to fahrenheit.
    
   - Which month has the most rain, on average?
    
   - Which year was the windiest?
    
   - What is the most frequent type of weather in January?
    
   - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    
   - What percentage of days were rainy in q3 of 2015?
    
   - For each year, find what percentage of days it rained (had non-zero precipitation)

In [1]:
# imports
import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()